In [9]:
import numpy as np
import gymnasium as gym
import random
import math
env = gym.make("Taxi-v3", render_mode="human")
# change-able parameters:
discount_factor = 0.8
delta_threshold = 0.00001
epsilon = 1

In [10]:
def value_iteration(env, gamma, epsilon):
    """
    value iteration to find the optimal value function and policy.

    Args:
        env: The Taxi-v3 environment.
        gamma: Discount factor for future rewards.
        epsilon: Threshold for convergence - stop when changes are smaller than this.

    Returns:
        policy: The optimal policy for each state.
        V: The optimal value function.
    """
    num_states = env.observation_space.n  #  500 states in Taxi-v3
    num_actions = env.action_space.n  # and 6 possible actions

    # start with a value function of all zeros
    V = np.zeros(num_states)

    # keep iterating until value function converges
    while True:
        delta = 0  # track the biggest change in V to check for convergence
        # loop through each state
        for s in range(num_states):
            v_old = V[s]  # track old value
            #compute value for each action in the state
            action_values = np.zeros(num_actions)
            for a in range(num_actions):
                # Get transition probabilities for state-action pair
                transitions = env.unwrapped.P[s][a]  # List of (prob, next_state, reward, done)
                for prob, next_state, reward, done in transitions:
                    # Add the expected value
                    action_values[a] += prob * (reward + gamma * V[next_state])
            # Update V(s) with the best action's value (Bellman optimality equation)
            V[s] = np.max(action_values)
            # Update delta to track the biggest change in V
            delta = max(delta, abs(v_old - V[s]))

        # check if converged - if the biggest change is small enough, then v has converged
        if delta < epsilon:
            break

    # extract optimal policy
    policy = np.zeros(num_states, dtype=int)
    for s in range(num_states):
        # compute the value for each action to find the best one
        action_values = np.zeros(num_actions)
        for a in range(num_actions):
            transitions = env.unwrapped.P[s][a]
            for prob, next_state, reward, done in transitions:
                action_values[a] += prob * (reward + gamma * V[next_state])
        # best action state =the one with the highest value
        policy[s] = np.argmax(action_values)

    return policy, V

In [11]:
# Run value iteration
policy, V = value_iteration(env, discount_factor, delta_threshold)
# resetting the environment and executing the policy
state = env.reset()
#state = state[0]
step = 0
done = False
state = state[0]
max_steps = 100
for step in range(max_steps):
  # Getting max value against that state, so that we choose that action
  action = policy[state]
  new_state, reward, done, truncated, info = env.step(action) # information after taking the action
  env.render()
  if done:
    print("number of steps taken:", step)
    break
  state = new_state
env.close()


number of steps taken: 11
